# Analyzing CIA Factbook Data Using SQL

## Introduction

In this project we are going to analyze some data from the [CIA World](https://www.cia.gov/library/publications/the-world-factbook/) Factbook that contains compendium of statistics about all the countries on earth. 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [2]:
%%sql
SELECT *
    FROM sqlite_master
    WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here is a description of all the columns in the database:
- name - The name of the country.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.

## Summary Statistics

In [5]:
%%sql
SELECT MIN(population) AS min_pop, 
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop,
       MAX(population_growth) AS max_pop
    FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_1,max_pop_1
0,7256490011,0.0,4.02


It is suspecious that the minimum population for at least a country is 0 and the maximum population for a country is more than 7.2 billion.

## Exploring Outliers

In [9]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MAX(population)
                            FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The database included a row including data for the whole world which indeed has a population of around 7.2 billion. We need to exclude this row in further analysis.

In [7]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MIN(population)
                            FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


The data for Antartica is correct given by the Factbook page on [Antartica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html).

## Recompute Summary Statistics 

In [15]:
%%sql
SELECT MIN(population) AS min_pop, 
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop,
       MAX(population_growth) AS max_pop
        FROM facts
        WHERE name <> 'World'

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_1,max_pop_1
0,1367485388,0.0,4.02


## Calculate Averages

In [16]:
%%sql
SELECT AVG(population) AS avg_population,
       AVG(area) AS avg_area
    FROM facts
    WHERE name <> 'World'

 * sqlite:///factbook.db
Done.


avg_population,avg_area
32242666.56846473,555093.546184739


## Explore Densely Populated Countries

In [12]:
%%sql
SELECT *
    FROM facts
    WHERE population > (SELECT AVG(population)
                           FROM facts) 
    AND area < (SELECT AVG(area)
                   FROM facts)
    ORDER BY population DESC;
    

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54


In the above table we listed the countries that are most dense in terms of population. 

## Country With Most People and Highest Growth Rate

In [18]:
%%sql 
SELECT name, MAX(population) AS max_pop
    FROM facts
    WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


name,max_pop
China,1367485388


In [20]:
%%sql
SELECT name, MAX(population_growth) AS max_growth_rate
    FROM facts
    WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


name,max_growth_rate
South Sudan,4.02


## Exploring Water and Land Data

In [25]:
%%sql
SELECT name, CAST(area_water AS FLOAT)/area_land AS water_land_ratio
    FROM facts
    WHERE water_land_ratio = (SELECT MAX(CAST(area_water AS FLOAT)/area_land) AS water_land_ratio
                                 FROM facts);

 * sqlite:///factbook.db
Done.


name,water_land_ratio
British Indian Ocean Territory,905.6666666666666


In [26]:
%%sql 
SELECT name, area_water, area_land
    FROM facts
    WHERE area_water > area_land

 * sqlite:///factbook.db
Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


## Exploring Birth Rate and Death Rate Data

In [28]:
%%sql
SELECT name, birth_rate
    FROM facts
    WHERE birth_rate = (SELECT MAX(birth_rate)
                           FROM facts);

 * sqlite:///factbook.db
Done.


name,birth_rate
Niger,45.45


In [29]:
%%sql 
SELECT name, birth_rate, death_rate
    FROM facts
    WHERE death_rate > birth_rate;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


## Explore Population to Area Ratio

In [31]:
%%sql
SELECT name, ROUND(CAST(population AS FLOAT)/area, 2) AS pop_area_ratio
    FROM facts
    ORDER BY pop_area_ratio DESC;

 * sqlite:///factbook.db
Done.


name,pop_area_ratio
Macau,21168.96
Monaco,15267.5
Singapore,8141.28
Hong Kong,6445.04
Gaza Strip,5191.82
Gibraltar,4876.33
Bahrain,1771.86
Maldives,1319.64
Malta,1310.02
Bermuda,1299.93
